In [1]:
import psycopg2

In [2]:
# Create your connection to the postgres database
conn = psycopg2.connect("host=127.0.0.1 dbname=studentdb user=postgres password=ramneekc")
cur = conn.cursor()
conn.set_session(autocommit=True)

Let's start with our normalized (3NF) database with the following set of tables
Table Name: transactions2 
column 0: transaction Id
column 1: Customer Name
column 2: Cashier Id
column 3: Year

Table Name: albums_sold
column 0: Album Id
column 1: Transaction Id
column 3: Album Name

Table Name: employees
column 0: Employee Id
column 1: Employee Name

Table Name: sales
column 0: Transaction Id
column 1: Amount Spent

In [3]:
# Create tables
cur.execute("CREATE TABLE IF NOT EXISTS transactions2 (transaction_id int, customer_name text, cashier_id int, \
                                                            year int)")
cur.execute("CREATE TABLE IF NOT EXISTS albums_sold (album_id int, transaction_id int, album_name text)")

cur.execute("CREATE TABLE IF NOT EXISTS employees (employee_id int, employee_name text)")

cur.execute("CREATE TABLE IF NOT EXISTS sales (transaction_id int, amount_spent int)")
# Insert data into the tables 

cur.execute("INSERT INTO transactions2 (transaction_id, customer_name, cashier_id, year) \
                 VALUES (%s, %s, %s, %s)", \
                 (1, "Amanda", 1, 2000))
cur.execute("INSERT INTO transactions2 (transaction_id, customer_name, cashier_id, year) \
                 VALUES (%s, %s, %s, %s)", \
                 (2, "Toby", 1, 2000))
cur.execute("INSERT INTO transactions2 (transaction_id, customer_name, cashier_id, year) \
                 VALUES (%s, %s, %s, %s)", \
                 (3, "Max", 2, 2018))

cur.execute("INSERT INTO albums_sold (album_id, transaction_id, album_name) \
                 VALUES (%s, %s, %s)", \
                 (1, 1, "Rubber Soul"))
cur.execute("INSERT INTO albums_sold (album_id, transaction_id, album_name) \
                 VALUES (%s, %s, %s)", \
                 (2, 1, "Let It Be"))
cur.execute("INSERT INTO albums_sold (album_id, transaction_id, album_name) \
                 VALUES (%s, %s, %s)", \
                 (3, 2, "My Generation"))
cur.execute("INSERT INTO albums_sold (album_id, transaction_id, album_name) \
                 VALUES (%s, %s, %s)", \
                 (4, 3, "Meet the Beatles"))
cur.execute("INSERT INTO albums_sold (album_id, transaction_id, album_name) \
                 VALUES (%s, %s, %s)", \
                 (5, 3, "Help!"))

cur.execute("INSERT INTO employees (employee_id, employee_name) \
                 VALUES (%s, %s)", \
                 (1, "Sam"))
cur.execute("INSERT INTO employees (employee_id, employee_name) \
                 VALUES (%s, %s)", \
                 (2, "Bob"))

cur.execute("INSERT INTO sales (transaction_id, amount_spent) \
                 VALUES (%s, %s)", \
                 (1, 40))
cur.execute("INSERT INTO sales (transaction_id, amount_spent) \
                 VALUES (%s, %s)", \
                 (2, 19))
cur.execute("INSERT INTO sales (transaction_id, amount_spent) \
                 VALUES (%s, %s)", \
                 (3, 45))

In [4]:
print("Table: transactions2\n")
try: 
    cur.execute("SELECT * FROM transactions2;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

print("\nTable: albums_sold\n")
try: 
    cur.execute("SELECT * FROM albums_sold;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

print("\nTable: employees\n")
try: 
    cur.execute("SELECT * FROM employees;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()
    
print("\nTable: sales\n")
try: 
    cur.execute("SELECT * FROM sales;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

Table: transactions2

(1, 'Amanda', 1, 2000)
(2, 'Toby', 1, 2000)
(3, 'Max', 2, 2018)

Table: albums_sold

(1, 1, 'Rubber Soul')
(2, 1, 'Let It Be')
(3, 2, 'My Generation')
(4, 3, 'Meet the Beatles')
(5, 3, 'Help!')

Table: employees

(1, 'Sam')
(2, 'Bob')

Table: sales

(1, 40)
(2, 19)
(3, 45)


Performed a 3 way JOIN on the 4 tables created

In [5]:
try: 
    cur.execute("SELECT t2.transaction_id, e.employee_name, t2.year, al.album_name, s.amount_spent \
                 FROM transactions2 t2 JOIN albums_sold al ON t2.transaction_id = al.transaction_id \
                 JOIN employees e ON t2.cashier_id = e.employee_id \
                 JOIN sales s ON t2.transaction_id = s.transaction_id")
    
    
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

(1, 'Sam', 2000, 'Rubber Soul', 40)
(1, 'Sam', 2000, 'Let It Be', 40)
(2, 'Sam', 2000, 'My Generation', 19)
(3, 'Bob', 2018, 'Meet the Beatles', 45)
(3, 'Bob', 2018, 'Help!', 45)


But, we had to perform a 3 way JOIN to get there. While it's great we had that flexibility, we need to remember that JOINS are slow and if we have a read heavy workload that required low latency queries we want to reduce the number of JOINS. Let's think about denormalizing our normalized tables.

Denormalize to so we can perform a query that should generate the amount spent on each transaction

One way to do this would be to do a JOIN on the sales and transactions2 table but we want to minimize the use of JOINS.

To reduce the number of tables, first add amount_spent to the transactions table so that you will not need to do a JOIN at all.